In [ ]:
# Entorno y gestion de paquetes
!pip install spacy --quiet
#!pip spacy download es_core_news_sm --quiet
!python -m spacy download es_core_news_md --quiet
!python -m spacy download es_core_news_sm --quiet

!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev --quiet
!pip install pdftotext --quiet
!pip install PyPDF2 --quiet
!pip install pdfminer.six --quiet

In [2]:
# Carga librerias
import warnings
warnings.filterwarnings('ignore')
import spacy
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress
from nltk import word_tokenize
import re
from IPython.display import Image
import PyPDF2
import csv
import pdftotext
import PyPDF2
from string import digits
import os
import pdftotext
import nltk
import string
import re
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
esp_stop = stopwords.words('spanish')
nlp = spacy.load("es_core_news_sm")

In [ ]:
import pandas as pd

# Read an Excel file into a DataFrame
df = pd.read_excel('/content/drive/MyDrive/NA&SB/NCG 461/Listado empresas 2023.xlsx')

In [6]:
import time
import requests

def download_with_wget(df, urls_column, filename_column):
    """Downloads files using the wget command and renames them based on the DataFrame column

    Arguments:
        df (pandas.DataFrame): The DataFrame containing the columns with URLs and filenames
        urls_column (str): The name of the column with URLs
        filename_column (str): The name of the column with filenames

    Returns:
        None
    """
    # Filter only the rows where urls_column has text
    filtered_df = df[df['Link'].notnull()]
    if not filtered_df.empty:
        for url, filename in zip(filtered_df[urls_column], filtered_df[filename_column]):
            try:
                start = time.time()
                response = requests.get(url, stream=True, timeout=5)
                response.raise_for_status()
                with open(f'{filename}.pdf', 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f'{filename}.pdf downloaded successfully')
            except requests.exceptions.Timeout:
                print(f'Timeout downloading {filename}.pdf')
            except requests.exceptions.HTTPError as err:
                print(f'HTTP error {err} while downloading {filename}.pdf')
            except Exception as e:
                print(f'Error {e} while downloading {filename}.pdf')
    else:
        print('No URLs with text found in the urls_column')


In [7]:
pdf_directory = "/content/drive/MyDrive/NA&SB/Memorias/pdf"
text_directory = "/content/drive/MyDrive/NA&SB/Memorias/txt"
punctuations1 = '''!()-[]{}"#$&'*+,./;:<=>?@\^_`|"~“—”¬ø¿•º–°'''
stopword_es = nltk.corpus.stopwords.words('spanish')
digits = string.digits

def procesar_texto(texto):
    texto = texto.lower()
    text_tokens = word_tokenize(texto)
    tokens_without_sw = [word for word in text_tokens if not word in stopword_es]
    filtered_text = (" ").join(tokens_without_sw)
    text_tokens = word_tokenize(filtered_text)
    propuesta_filtrada = (" ").join(tokens_without_sw)
    propuesta_filtrada = propuesta_filtrada.translate(str.maketrans('', '', string.punctuation))
    propuesta_filtrada = propuesta_filtrada.translate(str.maketrans('', '', punctuations1))
    propuesta_filtrada = propuesta_filtrada.translate(str.maketrans('', '', digits))
    propuesta_filtrada = re.sub('\n\n+', '\n', propuesta_filtrada)
    propuesta_filtrada = re.sub('\n', '', propuesta_filtrada)
    propuesta_filtrada = re.sub('(?<=\w)\s*\n(?=\w)', ' ', propuesta_filtrada)
    propuesta_filtrada = propuesta_filtrada.replace("  ", " ")
    propuesta_filtrada = propuesta_filtrada.replace("  ", " ")
    propuesta_filtrada = propuesta_filtrada.strip()
    propuesta_filtrada = propuesta_filtrada.rstrip()
    return propuesta_filtrada

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_filepath = os.path.join(pdf_directory, filename)
        with open(pdf_filepath, "rb") as f:
            pdf = pdftotext.PDF(f)
        text = "\n\n".join(pdf)
        processed_text = procesar_texto(text)
        text_filepath = os.path.join(text_directory, os.path.splitext(filename)[0] + '.txt')
        with open(text_filepath, 'w') as f:
            f.write(processed_text)


In [8]:
import pandas as pd

dicc = '/content/drive/MyDrive/NA&SB/NCG 461/ESG diccionario.xlsx'

df = pd.read_excel(dicc , header=1)
df=df[['E', 'S', 'G']]
cols = df.columns
lists = [df[col].dropna().tolist() for col in cols]
lists = [list(filter(lambda x: not pd.isna(x), sublist)) for sublist in lists]
lists = [[procesar_texto(text) for text in sublist] for sublist in lists]

In [9]:
E=lists[0]
S=lists[1]
G=lists[2]

In [ ]:
import collections
import os
import pandas as pd

lists = [E, S, G]
list_names = ['E', 'S', 'G']
folder_path = "/content/drive/MyDrive/NA&SB/Memorias/txt/"

data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            text = file.read()

        text_words = text.split()
        word_counts = collections.Counter(text_words)
        total_words = len(text_words)

        total = 0
        row = {'File': filename.replace('.txt', ''), 'Total Words': total_words}
        for i in range(len(lists)):
            current_list = lists[i]
            current_list_name = list_names[i]
            count = sum([word_counts[word] for word in current_list])
            total += count
            row[current_list_name] = count
            print(f"The total number of occurrences of the words in {current_list_name} in the file {filename.replace('.txt', '')} is {count}.")
        row['Total ESG'] = total
        data.append(row)
        print(f"The total number of occurrences of all words in all lists in the file {filename.replace('.txt', '')} is {total}.")
        print(f"The total number of words in the file {filename.replace('.txt', '')} is {total_words}.")

df = pd.DataFrame(data)
df = df[['File', 'E', 'S', 'G', 'Total ESG', 'Total Words']]
df['ESG Score']=100*df['Total ESG']/df['Total Words']
df = df.sort_values("ESG Score", ascending=False)
df['Rank'] = range(1, len(df) + 1)
df.to_excel("/content/drive/MyDrive/NA&SB/NCG 461/Ranking_ESGV1.xlsx", index=False)
